In [1]:
:!./install.sh

## module PiCalc

In [2]:
{-# LANGUAGE FlexibleContexts          #-}
{-# LANGUAGE FlexibleInstances         #-}
{-# LANGUAGE MultiParamTypeClasses     #-}
{-# LANGUAGE NoMonomorphismRestriction #-}
{-# LANGUAGE ScopedTypeVariables       #-}
{-# LANGUAGE DeriveGeneric             #-}
{-# LANGUAGE UndecidableInstances      #-}

module PiCalc where
import GHC.Generics (Generic)
import Data.Maybe
import Data.List
import Unbound.Generics.LocallyNameless

type Nm = Name Tm
newtype Tm = Var Nm deriving (Eq, Ord, Show, Generic)

data Pr  = Null | TauP Pr | Out Tm Tm Pr | In Tm PrB | Match Tm Tm Pr
         | Plus Pr Pr | Par Pr Pr | Nu PrB  deriving (Eq, Ord, Show, Generic)
type PrB = Bind Nm Pr
instance Eq PrB where (==) = aeq
instance Ord PrB where compare = acompare

data Act   = Up Tm Tm  | Tau     deriving (Eq, Ord, Show, Generic)
data ActB  = UpB Tm    | DnB Tm  deriving (Eq, Ord, Show, Generic)

data Form  = FF | TT | Conj [Form] | Disj [Form]
           | Dia  Act Form  |  DiaB  ActB FormB   | DiaMatch [(Tm,Tm)] Form
           | Box  Act Form  |  BoxB  ActB FormB   | BoxMatch [(Tm,Tm)] Form
           deriving (Eq, Ord, Show, Generic)
type FormB = Bind Nm Form
instance Eq FormB where (==) = aeq
instance Ord FormB where compare = acompare

instance Alpha Tm; instance Alpha Act; instance Alpha ActB
instance Alpha Pr; instance Alpha Form

instance Subst Tm Tm where isvar (Var x) = Just (SubstName x)
instance Subst Tm Act;  instance Subst Tm ActB
instance Subst Tm Pr;  instance Subst Tm Form

infixr 1 .\
(.\) = bind

x .= y = Match (Var x) (Var y)
inp = In . Var
out x y = Out(Var x)(Var y)
tau = TauP Null
tautau = TauP (TauP Null)

conj  = cn . filter(/=TT) where cn  [] = TT; cn  [f] = f; cn  fs = Conj fs
disj  = ds . filter(/=FF) where ds  [] = FF; ds  [f] = f; ds  fs = Disj fs

unbind2' b1 b2 = do  Just (x,p1,_,p2) <- unbind2 b1 b2
                     return (x,p1,p2)
(.+)  = Plus  ;   infixl 6 .+
(.|)  = Par   ;  infixl 5 .|
o = Null
taup = TauP
nu = Nu


{- To port these, may need DeriveDataTypeable
------------------------------------------------------------------
-- transformation/reduction of processes via generic programming
------------------------------------------------------------------
removeNull :: Rep a => a -> a
removeNull a = case cast a of
  Just(Plus Null x) -> fromJust(cast x)
  Just(Plus x Null) -> fromJust(cast x)
  Just(Par Null x) -> fromJust(cast x)
  Just(Par x Null) -> fromJust(cast x)
  _ -> a
-- rotate right for associative operators Plus and Par
rotateRight :: Rep a => a -> a
rotateRight a = case cast a of
  Just(Plus (Plus x y) z) -> fromJust . cast $ Plus x (Plus y z)
  Just(Par (Par x y) z) -> fromJust . cast $ Par x (Par y z)
  _ -> a
-- nub/sort for commutative operators Plus and Par
nubSortComm :: Rep a => a -> a
nubSortComm a = case cast a of
  Just p@(Plus _ _) -> fromJust . cast . foldr1 Plus . nubSort $
    unfoldr (\q -> case q of { Plus x y -> Just(x,y) ; _ -> Nothing }) p
  Just p@(Par _ _) -> fromJust . cast . foldr1 Par . nubSort $
    unfoldr (\q -> case q of { Par x y -> Just(x,y) ; _ -> Nothing }) p
  _ -> a
simplify = everywhere nubSortComm
         . everywhere rotateRight
         . everywhere removeNull
{-
foldl1 Plus (replicate 3 $ foldl1 Par [Null,Null,Null])
everywhere rotateRight $ foldl1 Plus (replicate 3 $ foldl1 Par [Null,Null,Null])
red $ foldl1 Plus (replicate 3 $ foldl1 Par [Null,Null,Null])
-}
-}

## module MemoUgly

In [3]:
module MemoUgly where
import Control.Concurrent.MVar
import qualified Data.Map as M
import System.IO.Unsafe(unsafePerformIO)
import qualified Data.MemoUgly

memo f = Data.MemoUgly.memo f
memoIO f = Data.MemoUgly.memoIO f

memoFix ff = f where f = memo (ff f)

Line 7: Eta reduce
Found:
memo f = Data.MemoUgly.memo f
Why not:
memo = Data.MemoUgly.memoLine 8: Eta reduce
Found:
memoIO f = Data.MemoUgly.memoIO f
Why not:
memoIO = Data.MemoUgly.memoIO

In [4]:
{-# LANGUAGE FlexibleContexts          #-}
{-# LANGUAGE FlexibleInstances         #-}
{-# LANGUAGE MultiParamTypeClasses     #-}
{-# LANGUAGE NoMonomorphismRestriction #-}
{-# LANGUAGE ScopedTypeVariables       #-}
{-# LANGUAGE UndecidableInstances      #-}

module IdSubLTS where
import           Control.Applicative
import           Control.Monad
import           Control.Monad.Fail
import           PiCalc
import           Unbound.Generics.LocallyNameless

-- one :: (Fresh m, Alternative m, MonadFail m) => Pr -> m (Act, Pr)
one (Out x y p)    = return (Up x y, p)
one (TauP p)       = return (Tau, p)
one (Match x y p)  | x == y = one p
one (Plus p q) = one p <|> one q
one (Par p q)
  =    do  (l, p') <- one p;  return (l, Par p' q)
  <|>  do  (l, q') <- one q;  return (l, Par p q')
  <|>  do  (lp, bp) <- oneb p;  (lq, bq) <- oneb q
           case (lp, lq) of  (UpB x,DnB x') | x == x'           -- close
                                            -> do  (y, p', q') <- unbind2' bp bq
                                                   return (Tau, Nu(y.\Par p' q'))
                             (DnB x',UpB x) | x' == x           -- close
                                            -> do  (y, q', p') <- unbind2' bq bp
                                                   return (Tau, Nu(y.\Par p' q'))
                             _              -> empty
  <|>  do  (Up x v, p') <- one p;  (DnB x', (y,q')) <- oneb' q
           guard $ x == x'
           return (Tau, Par p' (subst y v q'))  -- interaction
  <|>  do  (DnB x', (y,p')) <- oneb' p;  (Up x v, q') <- one q
           guard $ x == x'
           return (Tau, Par (subst y v p') q')  -- interaction
one (Nu b)  = do  (x,p) <- unbind b
                  (l,p') <- one p
                  case l of  Up (Var x') (Var y)  | x == x'  -> empty
                                                  | x == y   -> empty
                             _                    -> return (l, Nu (x.\p'))
one _       = empty

-- oneb :: (Fresh m, Alternative m, MonadFail m) => Pr -> m (ActB, PrB)
oneb (In x p)      = return (DnB x, p)
oneb (Match x y p) | x == y = oneb p
oneb (Plus p q)  = oneb p <|> oneb q
oneb (Par p q)   =     do  (l,(x,p')) <- oneb' p;  return (l, x.\Par p' q)
                 <|>   do  (l,(x,q')) <- oneb' q;  return (l, x.\Par p q')
oneb (Nu b)      =     do  (x,p) <- unbind b
                           (l,(y,p')) <- oneb' p
                           case l of  UpB (Var x')   | x == x' -> empty
                                      DnB (Var x')   | x == x' -> empty
                                      _              -> return (l, y.\Nu (x.\p'))
                 <|>   do  (x,p) <- unbind b
                           (Up y (Var x'),p') <- one p
                           guard $ x == x' && Var x /= y
                           return (UpB y, x.\p')  -- open
oneb _           = empty

oneb' p = do (l,b) <- oneb p; r <- unbind b; return (l,r)

{-
% Finite pi-calculus specification in lambda-Prolog
% A specification of the late transition system for the finite pi calculus.
% bound input
oneb (in X M) (dn X) M.
% free output
one (out X Y P) (up X Y) P.
% tau
one  (taup P) tau P.
% match prefix
one  (match X X P) A Q :- one  P A Q.
oneb (match X X P) A M :- oneb P A M.
% sum
one  (plus P Q) A R :- one  P A R.
one  (plus P Q) A R :- one  Q A R.
oneb (plus P Q) A M :- oneb P A M.
oneb (plus P Q) A M :- oneb Q A M.
% par
one  (par P Q) A (par P1 Q) :- one P A P1.
one  (par P Q) A (par P Q1) :- one Q A Q1.
oneb (par P Q) A (x\par (M x) Q) :- oneb P A M.
oneb (par P Q) A (x\par P (N x)) :- oneb Q A N.
% restriction
one  (nu x\P x) A (nu x\Q x)      :- pi x\ one  (P x) A (Q x).
oneb (nu x\P x) A (y\ nu x\Q x y) :- pi x\ oneb (P x) A (y\ Q x y).
% open
oneb (nu x\M x) (up X) N :- pi y\ one (M y) (up X y) (N y).
% close
one (par P Q) tau (nu y\ par (M y) (N y)) :- oneb P (dn X) M , oneb Q (up X) N.
one (par P Q) tau (nu y\ par (M y) (N y)) :- oneb P (up X) M , oneb Q (dn X) N.
% comm
one (par P Q) tau (par (M Y) T) :-  oneb P (dn X) M, one Q (up X Y) T.
one (par P Q) tau (par R (M Y)) :-  oneb Q (dn X) M, one P (up X Y) R.
-}


## module OpenLTS

In [5]:
{-# LANGUAGE FlexibleContexts          #-}
{-# LANGUAGE FlexibleInstances         #-}
{-# LANGUAGE MultiParamTypeClasses     #-}
{-# LANGUAGE NoMonomorphismRestriction #-}
{-# LANGUAGE ScopedTypeVariables       #-}
{-# LANGUAGE UndecidableInstances      #-}
{-# LANGUAGE DeriveGeneric             #-}

module OpenLTS where
import           GHC.Generics (Generic)
import           Control.Applicative
import           Control.Lens.Fold
import           Control.Monad
import           Control.Monad.Fail
import           Data.Map.Strict         (Map (..), fromList, insert, (!))
import           Data.Partition          hiding (empty, rep)
import qualified Data.Partition          as P
import           PiCalc
import           Unbound.Generics.LocallyNameless hiding (fv)
import qualified Unbound.Generics.LocallyNameless as U
{-# ANN module "HLint: ignore Use mappend" #-}
{-# ANN module "HLint: ignore Use camelCase" #-}

fv = toListOf U.fv

type EqC = [(Nm,Nm)]
{-
infixr 5 .:
(.:) :: (Nm,Nm) -> EqC -> EqC
(x,y) .: sigma = case compare x y of  LT -> [(x,y)] .++ sigma
                                      EQ -> sigma
                                      GT -> [(y,x)] .++ sigma
infixr 5 .++
(.++) = union
-}

type Ctx = [Quan]
data Quan = All Nm | Nab Nm deriving (Eq, Ord, Show, Generic)
quan2nm :: Quan -> Nm
quan2nm (All x) = x
quan2nm (Nab x) = x

instance Alpha Quan
instance Subst Tm Quan


-- wrapper -------------------
one nctx p = do (sigma, r) <- one_ ctx p
                return (part2eqc ctx sigma, r)
  where ctx = toCtx' nctx
oneb nctx b = do (sigma, r) <- one_b ctx b
                 return (part2eqc ctx sigma, r)
  where ctx = toCtx' nctx
-----------------------------

type Ctx' = (Ctx, Int, Map Nm Int)
emptyCtx' = ([], 0, fromList [])
type EqC' = Partition Int

toCtx' :: Ctx -> Ctx'
toCtx' nctx = (nctx, maxVal, n2iMap)
  where
    revns = reverse (fv nctx)
    n2iMap = fromList $ zip revns [0..maxVal]
    maxVal = length nctx - 1

extend :: Quan -> Ctx' -> Ctx'
extend q (nctx, n, n2iMap) = (q:nctx, n+1, insert (quan2nm q) n n2iMap)

respects :: EqC -> Ctx -> Bool
respects sigma nctx = all (\n -> P.rep part n == n) [n2i x | Nab x <- nctx]
  where (part, (n2i, _)) = mkPartitionFromEqC nctx sigma

respects' :: EqC -> Ctx' -> Bool
respects' sigma ctx@(nctx,_,n2iMap) =
  all (\n -> P.rep part n == n) [n2i x | Nab x <- nctx]
  where (part,(n2i,i2n)) = mkPartitionFromEqC' ctx sigma

respects_ :: EqC' -> Ctx' -> Bool
respects_ sigma ctx@(nctx,_,n2iMap) =
  all (\n -> P.rep sigma n == n) [n2i x | Nab x <- nctx]
  where (n2i,i2n) = mkMapFunsFromEqC' ctx sigma

subs :: Subst Tm b => Ctx -> EqC -> b -> b
subs nctx sigma = substs [(x,Var y) | (x,y)<-sigma']
  where  sigma' = [(i2n i, i2n $ P.rep part i) | i<-[0..maxVal]]
         (part, (n2i, i2n)) = mkPartitionFromEqC nctx sigma
         maxVal = length nctx - 1

subs_ :: Subst Tm b => Ctx' -> EqC' -> b -> b
subs_ ctx@(nctx,maxVal,n2iMap) sigma = substs [(x,Var y) | (x,y)<-sigma']
  where  sigma' = [(i2n i, i2n $ P.rep sigma i) | i<-[0..maxVal]]
         (n2i, i2n) = mkMapFunsFromEqC' ctx sigma

mkPartitionFromEqC ::  Ctx -> EqC -> (Partition Int, (Nm -> Int, Int -> Nm))
mkPartitionFromEqC nctx sigma = (part, (n2i, i2n))
  where
    part =  foldr (.) id [joinElems (n2i x) (n2i y) | (x,y) <- sigma] discrete
    i2n i = revns !! i
    n2i x = n2iMap ! x
    revns = reverse (fv nctx)
    n2iMap = fromList $ zip revns [0..maxVal]
    maxVal = length nctx - 1

mkPartitionFromEqC' :: Ctx' -> EqC -> (Partition Int, (Nm -> Int, Int -> Nm))
mkPartitionFromEqC' (nctx,maxVal,n2iMap) sigma = (part, (n2i, i2n))
  where
    part =  foldr (.) id [joinElems (n2i x) (n2i y) | (x,y) <- sigma] discrete
    i2n i = quan2nm $ nctx !! (maxVal - i)
    n2i x = n2iMap ! x

joinNm ctx (x,y) sigma = joinElems (n2i x) (n2i y) sigma
  where (n2i,i2n) = mkMapFunsFromEqC' ctx sigma

joinParts sigma_p sigma_q = fromSets $ nontrivialSets sigma_p ++ nontrivialSets sigma_q

mkMapFunsFromEqC' :: Ctx' -> EqC' -> (Nm -> Int, Int -> Nm)
mkMapFunsFromEqC' (nctx,maxVal,n2iMap) sigma = (n2i, i2n)
  where
    i2n i = quan2nm $ nctx !! (maxVal - i)
    n2i x = n2iMap ! x

part2eqc :: Ctx' -> EqC' -> EqC
part2eqc ctx@(nctx,maxVal,n2iMap) sigma =
  [(i2n i, i2n j) | (i,j) <- [(i, P.rep sigma i) | i<-[0..maxVal]], i/=j]
  where (n2i,i2n) = mkMapFunsFromEqC' ctx sigma

-- one_ :: (Fresh m, Alternative m) => Ctx' -> Pr -> m (EqC',(Act,Pr))
one_ ctx  (Out x y p)   = return (P.empty, (Up x y, p))
one_ ctx  (TauP p)      = return (P.empty, (Tau, p))
one_ ctx  (Match (Var x) (Var y) p)  | x == y                   = one_ ctx p
                                     | [(x,y)] `respects'` ctx  =
                                           do  (sigma, r) <- one_ ctx p
                                               let sigma' = joinNm ctx (x,y) sigma
                                               guard $ sigma' `respects_` ctx
                                               return (sigma', r)
one_ ctx  (Plus p q) = one_ ctx p <|> one_ ctx q
one_ ctx  (Par p q)
  =    do  (sigma,(l,p')) <- one_ ctx p;  return (sigma,(l,Par p' q))
  <|>  do  (sigma,(l,q')) <- one_ ctx q;  return (sigma,(l,Par p q'))
  <|>  do  (sigma_p,(lp,bp)) <- one_b ctx p;  (sigma_q,(lq,bq)) <- one_b ctx q
           case (lp, lq) of             -- close
             (DnB(Var x),UpB(Var x'))  -> do  (y, q', p') <-  unbind2' bq bp
                                              let sigma' = joinNm ctx (x,x') (joinParts sigma_p sigma_q)
                                              guard $ sigma' `respects_` ctx
                                              return (sigma', (Tau, Nu(y.\Par p' q')))
             (UpB(Var x'),DnB(Var x))  -> do  (y, p', q') <- unbind2' bp bq
                                              let sigma' = joinNm ctx (x,x') (joinParts sigma_p sigma_q)
                                              guard $ sigma' `respects_` ctx
                                              return (sigma', (Tau, Nu(y.\Par p' q')))
             _                         -> empty
  <|>  do  (sigma_p, (Up (Var x) v, p')) <- one_ ctx p
           (sigma_q, (DnB (Var x'), bq)) <- one_b ctx q;  (y, q') <- unbind bq
           let sigma' = joinNm ctx (x,x') (joinParts sigma_p sigma_q)
           guard $ sigma' `respects_` ctx
           return (sigma', (Tau, Par p' (subst y v q'))) -- interaction
  <|>  do  (sigma_p, (DnB (Var x'), (y, p')))   <- one_b'  ctx p
           (sigma_q, (Up (Var x) v,     q'))    <- one_    ctx q
           let sigma' = joinNm ctx (x,x') (joinParts sigma_p sigma_q)
           guard $ sigma' `respects_` ctx
           return (sigma', (Tau, Par (subst y v p') q'))
one_ ctx (Nu b) = do  (x,p) <- unbind b;              let ctx' = extend (Nab x) ctx
                      (sigma,(l,p')) <- one_ ctx' p;  let sigmaSubs = subs_ ctx' sigma
                      case l of  Up (Var x') (Var y)  | x == sigmaSubs x'  -> empty
                                                      | x == sigmaSubs y   -> empty
                                 _                    -> return (sigma, (l, Nu(x.\p')))
one_ _    _      = empty

-- one_b :: (Fresh m, Alternative m) => Ctx' -> Pr -> m (EqC',(ActB, PrB))
one_b ctx (In x p) = return (P.empty, (DnB x, p))
one_b ctx (Match (Var x) (Var y) p)  | x == y                   = one_b ctx p
                                     | [(x,y)] `respects'` ctx  =
                                           do  (sigma, r) <- one_b ctx p
                                               let sigma' = joinNm ctx (x,y) sigma
                                               guard $ sigma' `respects_` ctx
                                               return (sigma', r)
one_b ctx (Plus p q) = one_b ctx p <|> one_b ctx q
one_b ctx (Par p q) =
       do (sigma,(l,(x,p'))) <- one_b' ctx p;  return (sigma,(l, x.\Par p' q))
  <|>  do (sigma,(l,(x,q'))) <- one_b' ctx q;  return (sigma,(l, x.\Par p q'))
one_b ctx (Nu b)  =    do  (x,p) <- unbind b;                    let ctx' = extend (Nab x) ctx
                           (sigma,(l,(y,p'))) <- one_b' ctx' p;  let sigmaSubs = subs_ ctx' sigma
                           case l of  UpB (Var x')  | x == sigmaSubs x' -> empty
                                      DnB (Var x')  | x == sigmaSubs x' -> empty
                                      _             -> return (sigma, (l, y.\Nu (x.\p')))
                  <|>  do  (x,p) <- unbind b;                          let ctx' = extend (Nab x) ctx
                           (sigma,(Up y (Var x'),p')) <- one_ ctx' p;  let sigmaSubs = subs_ ctx' sigma
                           guard $ x == sigmaSubs x' && Var x /= sigmaSubs y
                           return (sigma, (UpB y, x.\p')) -- open
one_b _    _ = empty

one_b' ctx p = do (sigma,(l,b)) <- one_b ctx p; r <- unbind b; return (sigma,(l,r))

Line 63: Reduce duplication
Found:
revns = reverse (fv nctx)
n2iMap = fromList $ zip revns [0 .. maxVal]
maxVal = length nctx - 1

Why not:
Combine with <unknown>.hs:101:5Line 134: Reduce duplication
Found:
let sigma' = joinNm ctx (x, y) sigma
guard $ sigma' `respects_` ctx
return (sigma', r)

Why not:
Combine with <unknown>.hs:174:48Line 144: Reduce duplication
Found:
let sigma' = joinNm ctx (x, x') (joinParts sigma_p sigma_q)
guard $ sigma' `respects_` ctx
return (sigma', (Tau, Nu (y .\ Par p' q')))

Why not:
Combine with <unknown>.hs:148:47

## module OpenBisim

In [6]:
{-# LANGUAGE FlexibleContexts          #-}
{-# LANGUAGE FlexibleInstances         #-}
{-# LANGUAGE MultiParamTypeClasses     #-}
{-# LANGUAGE NoMonomorphismRestriction #-}
{-# LANGUAGE ScopedTypeVariables       #-}
{-# LANGUAGE UndecidableInstances      #-}
module OpenBisim where
import           Control.Applicative
import           Control.Lens.Fold
import           Control.Monad
import           Control.Monad.Fail
import qualified Control.Monad.Fail as Fail
import           Data.Tree
import qualified IdSubLTS
import           MemoUgly
import           OpenLTS                 hiding (one, oneb)
import           PiCalc
import           Unbound.Generics.LocallyNameless hiding (fv)
{-# ANN module "HLint: ignore Use mappend" #-}
 
instance MonadFail f => MonadFail (FreshMT f) where
  fail = runFreshMT . Fail.fail

data StepLog  =  One   Ctx EqC Act   Pr
              |  OneB  Ctx EqC ActB  PrB  deriving (Eq,Ord,Show)

returnL log = return . Node (Left log)   -- for the step on |p|'s side
returnR log = return . Node (Right log)  -- for the step on |q|'s side

-- wrapper -----------------------------
sim nctx = sim2 (toCtx' nctx)
sim' nctx = sim2' (toCtx' nctx)
bisim nctx = bisim2 (toCtx' nctx)
bisim' nctx = bisim2' (toCtx' nctx)
----------------------------------------

sim2 ctx p q = and $ sim2_ ctx p q

sim2_ :: Ctx' -> Pr -> Pr -> [Bool]
sim2_ ctx p q = memoFix sim2_unfix (ctx,p,q)

sim2_unfix f (ctx@(nctx,_,_), p, q)  =
       do  (sigma, r) <- runFreshMT (one_ ctx p); let sigmaSubs = subs_ ctx sigma
           let (lp, p') = sigmaSubs r
           return . (or :: [Bool] -> Bool) . runFreshMT $ do
             (lq, q') <-IdSubLTS.one (sigmaSubs q)
             guard $ lp == lq
             return . (and :: [Bool] -> Bool) $ f(ctx,p',q')
   <|>  do  (sigma, r) <- runFreshMT (one_b ctx p); let sigmaSubs = subs_ ctx sigma
            let (lp, bp') = sigmaSubs r
            let x' = runFreshM $ freshFrom (fv nctx) bp'
            return . (or :: [Bool] -> Bool) . runFreshMT $ do
              (lq, bq') <-IdSubLTS.oneb (sigmaSubs q)
              guard $ lp == lq
              (x, q1, p1) <- unbind2' bq' bp'
              let (p', q')  | x == x'    = (p1, q1)
                            | otherwise  = subst x (Var x') (p1, q1)
              let ctx' = case lp of   DnB _ -> extend (All x') ctx
                                      UpB _ -> extend (Nab x') ctx
              return . (and :: [Bool] -> Bool) $ f(ctx',p',q')

-- sim2' :: Ctx' -> Pr -> Pr -> [Tree (Either StepLog StepLog)]
sim2' ctx@(nctx,_,_) p q   =
        do   (sigma, r) <- runFreshMT (one_ ctx p); let sigmaSubs = subs_ ctx sigma
             let (lp, p') = sigmaSubs r
             returnL (One nctx (toEqC sigma) lp p') . runFreshMT $ do
               (lq, q') <-IdSubLTS.one (sigmaSubs q)
               guard $ lp == lq
               returnR (One nctx (toEqC sigma) lq q') $ sim2' ctx p' q'
  <|>   do   (sigma, r) <- runFreshMT (one_b ctx p); let sigmaSubs = subs_ ctx sigma
             let (lp, bp') = sigmaSubs r
             let x' = runFreshM $ freshFrom (fv nctx) bp'
             returnL (OneB nctx (toEqC sigma) lp bp') . runFreshMT $ do
               (lq, bq') <-IdSubLTS.oneb (sigmaSubs q)
               guard $ lp == lq
               (x, p1, q1) <- unbind2' bp' bq'
               let (p', q')   | x == x'    = (p1, q1)
                              | otherwise  = subst x (Var x') (p1, q1)
               let ctx' = case lp of   DnB _ -> extend (All x') ctx
                                       UpB _ -> extend (Nab x') ctx
               returnR (OneB nctx (toEqC sigma) lq bq') $ sim2' ctx' p' q'
  where toEqC = part2eqc ctx

bisim2 ctx p q = and $ bisim2_ ctx p q -- (simplify p) (simplify q)

bisim2_ ctx p q = memoFix bisim2_unfix (ctx,p,q)
bisim2_unfix f (ctx@(nctx,_,_),p,q) =
  do (sigma, r) <- runFreshMT (one_ ctx p)
     let (lp, p') = subs_ ctx sigma r
     return . (or :: [Bool] -> Bool) . runFreshMT $ do
       (lq, q') <-IdSubLTS.one (subs_ ctx sigma q) -- follow with same sub and label
       guard $ lp == lq
       return . (and :: [Bool] -> Bool) $ f(ctx,p',q')
  <|>
  do (sigma, r) <- runFreshMT (one_b ctx p)
     let (lp, bp') = subs_ ctx sigma r
     let x' = runFreshM $ freshFrom (fv nctx) bp' -- to use same new quan var
     return . (or :: [Bool] -> Bool) . runFreshMT $ do
       (lq, bq') <-IdSubLTS.oneb (subs_ ctx sigma q) -- follow with same sub and label
       guard $ lp == lq
       (x, p1, q1) <- unbind2' bp' bq'
       let (p', q') | x == x'   = (p1, q1) -- to use same new quan var
                    | otherwise = subst x (Var x') (p1, q1)
       let ctx' = case lp of DnB _ -> extend (All x') ctx
                             UpB _ -> extend (Nab x') ctx
       return . (and :: [Bool] -> Bool) $ f(ctx',p',q')
  <|>
  do (sigma, r) <- runFreshMT (one_ ctx q)
     let (lq, q') = subs_ ctx sigma r
     return . (or :: [Bool] -> Bool) . runFreshMT $ do
       (lp, p') <-IdSubLTS.one (subs_ ctx sigma p) -- follow with same sub and label
       guard $ lp == lq
       return . (and :: [Bool] -> Bool) $ f(ctx,p',q')
  <|>
  do (sigma, r) <- runFreshMT (one_b ctx q)
     let (lq, bq') = subs_ ctx sigma r
     let x' = runFreshM $ freshFrom (fv nctx) bq' -- to use same new quan var
     return . (or :: [Bool] -> Bool) . runFreshMT $ do
       (lp, bp') <-IdSubLTS.oneb (subs_ ctx sigma p) -- follow with same sub and label
       guard $ lp == lq
       (x, q1, p1) <- unbind2' bq' bp'
       let (p', q') | x == x'   = (p1, q1) -- to use same new quan var
                    | otherwise = subst x (Var x') (p1, q1)
       let ctx' = case lp of DnB _ -> extend (All x') ctx
                             UpB _ -> extend (Nab x') ctx
       return . (and :: [Bool] -> Bool) $ f(ctx',p',q')

bisim2' ctx@(nctx,_,_) p q =
  do (sigma, r) <- runFreshMT (one_ ctx p)
     let (lp, p') = subs_ ctx sigma r
     returnL (One nctx (toEqC sigma) lp p') . runFreshMT $ do
       (lq, q') <-IdSubLTS.one (subs_ ctx sigma q) -- follow with same sub and label
       guard $ lp == lq
       returnR (One nctx (toEqC sigma) lq q') $ bisim2' ctx p' q'
  <|>
  do (sigma, r) <- runFreshMT (one_b ctx p)
     let (lp, bp') = subs_ ctx sigma r
     let x' = runFreshM $ freshFrom (fv nctx) bp' -- to use same new quan var
     returnL (OneB nctx (toEqC sigma) lp bp') . runFreshMT $ do
       (lq, bq') <-IdSubLTS.oneb (subs_ ctx sigma q) -- follow with same sub and label
       guard $ lp == lq
       (x, p1, q1) <- unbind2' bp' bq'
       let (p', q') | x == x'   = (p1, q1) -- to use same new quan var
                    | otherwise = subst x (Var x') (p1, q1)
       let ctx' = case lp of DnB _ -> extend (All x') ctx
                             UpB _ -> extend (Nab x') ctx
       returnR (OneB nctx (toEqC sigma) lq bq') $ bisim2' ctx' p' q'
  <|>
  do (sigma, r) <- runFreshMT (one_ ctx q)
     let (lq, q') = subs_ ctx sigma r
     returnR (One nctx (toEqC sigma) lq q') . runFreshMT $ do
       (lp, p') <-IdSubLTS.one (subs_ ctx sigma p) -- follow with same sub and label
       guard $ lp == lq
       returnL (One nctx (toEqC sigma) lp p') $ bisim2' ctx p' q'
  <|>
  do (sigma, r) <- runFreshMT (one_b ctx q)
     let (lq, bq') = subs_ ctx sigma r
     let x' = runFreshM $ freshFrom (fv nctx) bq' -- to use same new quan var
     returnR (OneB nctx (toEqC sigma) lq bq') . runFreshMT $ do
       (lp, bp') <-IdSubLTS.oneb (subs_ ctx sigma p) -- follow with same sub and label
       guard $ lp == lq
       (x, q1, p1) <- unbind2' bq' bp'
       let (p', q') | x == x'   = (p1, q1) -- to use same new quan var
                    | otherwise = subst x (Var x') (p1, q1)
       let ctx' = case lp of DnB _ -> extend (All x') ctx
                             UpB _ -> extend (Nab x') ctx
       returnL (OneB nctx (toEqC sigma) lp bp') $ bisim2' ctx' p' q'
  where toEqC = part2eqc ctx

freshFrom :: Fresh m => [Nm] -> PrB -> m Nm
freshFrom xs b = do { mapM_ fresh xs; fst <$> unbind b }

forest2df :: [Tree (Either StepLog StepLog)] -> [(Form,Form)]
forest2df rs
            =    do  Node (Left (One _ sigma_p a _)) [] <- rs
                     let sigmaqs = subsMatchingAct a (right1s rs)
                     return (prebase sigma_p a, postbase sigmaqs a)
            <|>  do  Node (Right (One _ sigma_q a _)) [] <- rs
                     let formR = prebase sigma_q a
                     let sigmaps = subsMatchingAct a (left1s rs)
                     return (postbase sigmaps a, formR)
            <|>  do  Node (Left (OneB _ sigma_p a _)) [] <- rs
                     let sigmaqs = subsMatchingActB a (right1Bs rs)
                     return (preBbase sigma_p a, postBbase sigmaqs a)
            <|>  do  Node (Right (OneB _ sigma_q a _)) [] <- rs
                     let formR = preBbase sigma_q a
                     let sigmaps = subsMatchingActB a (left1Bs rs)
                     return (postBbase sigmaps a, formR)
            <|>  do  Node (Left (One _ sigma_p a _)) rsR <- rs
                     let rss' = [rs' | Node _ rs' <- rsR]
                     (dfsL,dfsR) <- unzip <$> sequence (forest2df <$> rss')
                     guard . not . null $ dfsL
                     let sigmaqs = subsMatchingAct a (right1s rs)
                     return (pre sigma_p a dfsL, post sigmaqs a dfsR)
            <|>  do  Node (Right (One _ sigma_q a _)) rsL <- rs
                     let rss' = [rs' | Node _ rs' <- rsL]
                     (dfsL,dfsR) <- unzip <$> sequence (forest2df <$> rss')
                     guard . not . null $ dfsL
                     let sigmaps = subsMatchingAct a (left1s rs)
                     return (post sigmaps a dfsL, pre sigma_q a dfsR)
            <|>  do  Node (Left (OneB nctx sigma_p a _)) rsR <- rs
                     let  rss' = [rs' | Node _ rs' <- rsR]
                          x = quan2nm . head . getCtx . fromEither
                            . rootLabel . head $ head rss'
                     (dfsL,dfsR) <- unzip <$> sequence (forest2df <$> rss')
                     guard . not . null $ dfsL
                     let sigmaqs = subsMatchingActB a (right1Bs rs)
                     return (preB sigma_p a x dfsL, postB sigmaqs a x dfsR)
            <|>  do  Node (Right (OneB nctx sigma_q a _)) rsL <- rs
                     let  rss' = [rs' | Node _ rs' <- rsL]
                          x = quan2nm . head . getCtx . fromEither . rootLabel
                                $ head (head rss')
                     (dfsL,dfsR) <- unzip <$> sequence (forest2df <$> rss')
                     guard . not . null $ dfsL
                     let sigmaps = subsMatchingActB a (left1Bs rs)
                     return (postB sigmaps a x dfsL, preB sigma_q a x dfsR)
  where
    prebase sigma a = pre sigma a []
    postbase sigmas a = post sigmas a []
    preBbase sigma a = preB sigma a (s2n "?") []
    postBbase sigmas a = postB sigmas a (s2n "?") []
    pre sigma a = boxMat sigma . Dia a . conj
    post sigmas a fs = Box a . disj $  (diaMat<$>sigmas) ++ fs
    preB sigma a x = boxMat sigma . DiaB a . bind x . conj
    postB sigmas a x fs = BoxB a . bind x . disj $  (diaMat<$>sigmas) ++ fs
    boxMat  [] = id; boxMat  sigma = BoxMatch [(Var x,Var y) | (x,y)<-sigma]
    diaMat  [] = FF; diaMat  sigma = DiaMatch [(Var x,Var y) | (x,y)<-sigma] TT
    right1s  rs = [log | Node (Right  log@One{}) _ <- rs]
    left1s   rs = [log | Node (Left   log@One{}) _ <- rs]
    right1Bs  rs = [log | Node (Right  log@OneB{}) _ <- rs]
    left1Bs   rs = [log | Node (Left   log@OneB{}) _ <- rs]
    getCtx (One   nctx _ _ _)  = nctx; getCtx (OneB  nctx _ _ _) = nctx
    fromEither (Left   t) = t; fromEither (Right  t) = t

subsMatchingAct :: Act -> [StepLog] -> [EqC]
subsMatchingAct a logs =
  do  One nctx sigma a' _ <-logs          ;  let sigmaSubs = subs nctx sigma
      guard $ sigmaSubs a == sigmaSubs a' ;  return sigma

subsMatchingActB :: ActB -> [StepLog] -> [EqC]
subsMatchingActB a logs =
  do  OneB nctx sigma a' _ <-logs         ;  let sigmaSubs = subs nctx sigma
      guard $ sigmaSubs a == sigmaSubs a' ;  return sigma

Line 43: Reduce duplication
Found:
(sigma, r) <- runFreshMT (one_ ctx p)
let sigmaSubs = subs_ ctx sigma
let (lp, p') = sigmaSubs r

Why not:
Combine with <unknown>.hs:64:14Line 49: Reduce duplication
Found:
(sigma, r) <- runFreshMT (one_b ctx p)
let sigmaSubs = subs_ ctx sigma
let (lp, bp') = sigmaSubs r
let x' = runFreshM $ freshFrom (fv nctx) bp'

Why not:
Combine with <unknown>.hs:70:14Line 54: Reduce duplication
Found:
guard $ lp == lq
(x, q1, p1) <- unbind2' bq' bp'
let (p', q')
      | x == x' = (p1, q1)
      | otherwise = subst x (Var x') (p1, q1)
let ctx'
      = case lp of
            DnB _ -> extend (All x') ctx
            UpB _ -> extend (Nab x') ctx
return . (and :: [Bool] -> Bool) $ f (ctx', p', q')

Why not:
Combine with <unknown>.hs:120:8Line 75: Reduce duplication
Found:
guard $ lp == lq
(x, p1, q1) <- unbind2' bp' bq'
let (p', q')
      | x == x' = (p1, q1)
      | otherwise = subst x (Var x') (p1, q1)
let ctx'
      = case lp of
            DnB _ -> extend (All x') ctx
            UpB _ -> extend (Nab x') ctx

Why not:
Combine with <unknown>.hs:100:8Line 95: Reduce duplication
Found:
(sigma, r) <- runFreshMT (one_b ctx p)
let (lp, bp') = subs_ ctx sigma r
let x' = runFreshM $ freshFrom (fv nctx) bp'

Why not:
Combine with <unknown>.hs:136:6Line 99: Reduce duplication
Found:
(lq, bq') <- IdSubLTS.oneb (subs_ ctx sigma q)
guard $ lp == lq
(x, p1, q1) <- unbind2' bp' bq'
let (p', q')
      | x == x' = (p1, q1)
      | otherwise = subst x (Var x') (p1, q1)
let ctx'
      = case lp of
            DnB _ -> extend (All x') ctx
            UpB _ -> extend (Nab x') ctx

Why not:
Combine with <unknown>.hs:140:8Line 115: Reduce duplication
Found:
(sigma, r) <- runFreshMT (one_b ctx q)
let (lq, bq') = subs_ ctx sigma r
let x' = runFreshM $ freshFrom (fv nctx) bq'

Why not:
Combine with <unknown>.hs:156:6Line 119: Reduce duplication
Found:
(lp, bp') <- IdSubLTS.oneb (subs_ ctx sigma p)
guard $ lp == lq
(x, q1, p1) <- unbind2' bq' bp'
let (p', q')
      | x == x' = (p1, q1)
      | otherwise = subst x (Var x') (p1, q1)
let ctx'
      = case lp of
            DnB _ -> extend (All x') ctx
            UpB _ -> extend (Nab x') ctx

Why not:
Combine with <unknown>.hs:160:8Line 237: Reduce duplication
Found:
let sigmaSubs = subs nctx sigma
guard $ sigmaSubs a == sigmaSubs a'
return sigma

Why not:
Combine with <unknown>.hs:242:46

# Examples

In [7]:
{-# LANGUAGE FlexibleContexts          #-}
{-# LANGUAGE FlexibleInstances         #-}
{-# LANGUAGE MultiParamTypeClasses     #-}
{-# LANGUAGE NoMonomorphismRestriction #-}
{-# LANGUAGE ScopedTypeVariables       #-}
{-# LANGUAGE UndecidableInstances      #-}

import Prelude hiding ((<>))

import           Data.Tree
import qualified IdSubLTS                       as IdS
import           OpenBisim
import qualified OpenLTS                        as OpS
import           PiCalc
import           Text.PrettyPrint
import           Text.PrettyPrint.HughesPJClass
import           Unbound.Generics.LocallyNameless

appPrec :: Rational
appPrec = 10

pp = print . pPrint

instance Pretty Nm where pPrint = text . show
instance Pretty Quan where
  pPrintPrec l r (All x) = maybeParens (r > appPrec) $ text "All" <+> ppp x
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (Nab x) = maybeParens (r > appPrec) $ text "Nab" <+> ppp x
    where ppp = pPrintPrec l (appPrec+1)
instance Pretty Tm where
  pPrintPrec l r (Var x) = maybeParens (r > appPrec) $ text "Var" <+> ppp x
    where ppp = pPrintPrec l (appPrec+1)
instance Pretty Act where
  pPrintPrec l r (Up x y) = maybeParens (r > appPrec) $ text "Up" <+> ppp x <+> ppp y
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r Tau = text "Tau"
instance Pretty ActB where
  pPrintPrec l r (UpB x) = maybeParens (r > appPrec) $ text "UpB" <+> ppp x
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (DnB x) = maybeParens (r > appPrec) $ text "DnB" <+> ppp x
    where ppp = pPrintPrec l (appPrec+1)
instance (Alpha a, Pretty a) => Pretty (Bind Nm a) where
  pPrintPrec l r b = maybeParens (r > appPrec) $ ppp x <> text ".\\" <> ppp p
    where ppp = pPrintPrec l (appPrec+1); (x,p) = runFreshM $ unbind b
instance Pretty Pr where
  pPrintPrec _ _ Null = text "Null"
  pPrintPrec l r (TauP p) = maybeParens (r > appPrec) $
            text "TauP" <+> ppp p
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (Out x y p) = maybeParens (r > appPrec) $
            text "Out" <+> ppp x <+> ppp y <+> ppp p
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (In x b) = maybeParens (r > appPrec) $
            text "In" <+> ppp x <+> ppp b
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (Plus p q) = maybeParens (r > appPrec) $
            ppp p <+> text "`Plus`" <+> ppp q
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (Par p q) = maybeParens (r > appPrec) $
            ppp p <+> text "`Par`" <+> ppp q
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (Nu b) = maybeParens (r > appPrec) $
            text "Nu" <> ppp b
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (Match x y p) = maybeParens (r > appPrec) $
            text "Match" <+> ppp x <+> ppp y <+> ppp p
    where ppp = pPrintPrec l (appPrec+1)
instance Pretty Form where
  pPrintPrec _ _ FF = text "FF"
  pPrintPrec _ _ TT = text "TT"
  pPrintPrec l r (Conj fs) = maybeParens (r > appPrec) $
            text "Conj" <> ppp fs
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (Disj fs) = maybeParens (r > appPrec) $
            text "Disj" <> ppp fs
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (Box a f) = maybeParens (r > appPrec) $
            text "Box" <+> ppp a <+> ppp f
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (Dia a f) = maybeParens (r > appPrec) $
             text "Dia" <+> ppp a <+> ppp f
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (BoxB a f) = maybeParens (r > appPrec) $
            text "BoxB" <+> ppp a <+> ppp f
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (DiaB a f) = maybeParens (r > appPrec) $
            text "DiaB" <+> ppp a <+> ppp f
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (BoxMatch sigma f) = maybeParens (r > appPrec) $
            text "BoxMatch" <+> ppp sigma <+> ppp f
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (DiaMatch sigma f) = maybeParens (r > appPrec) $
            text "DiaMatch" <+> ppp sigma <+> ppp f
    where ppp = pPrintPrec l (appPrec+1)
instance Pretty StepLog where
  pPrintPrec l r (One  nctx sigma a p) = maybeParens (r > appPrec) $
            text "One" <+> ppp nctx <+> ppp sigma <+> ppp a <+> ppp p
    where ppp = pPrintPrec l (appPrec+1)
  pPrintPrec l r (OneB nctx sigma a b) = maybeParens (r > appPrec) $
            text "OneB" <+> ppp nctx <+> ppp sigma <+> ppp a <+> ppp b
    where ppp = pPrintPrec l (appPrec+1)




nm2bdw :: Fresh m => Nm -> m String
nm2bdw x = return $ show x

tm2bdw :: Fresh m => Tm -> m String
tm2bdw (Var x) = nm2bdw x

act2bdw :: Fresh m => Act -> m String
act2bdw Tau      = return "tau"
act2bdw (Up x y) = (\x y -> "(up "++x++" "++y++")") <$> tm2bdw x <*> tm2bdw y

actb2bdw :: Fresh m => ActB -> m String
actb2bdw (UpB x) = (\x -> "(up "++x++")") <$> (tm2bdw x)
actb2bdw (DnB x) = (\x -> "(dn "++x++")") <$> (tm2bdw x)

pr2bdw :: Fresh m => Pr -> m String
pr2bdw Null = return "z"
pr2bdw (TauP p) = (\p->"(taup "++p++")") <$> pr2bdw p
pr2bdw (Out x y p) = (\x y p->"(out "++x++" "++y++" "++p++")")
                        <$> tm2bdw x <*> tm2bdw y <*> pr2bdw p
pr2bdw (In x b) = do (w,p) <- unbind b
                     (\x w p->"(in "++x++" "++w++"\\"++p++")")
                        <$> tm2bdw x <*> nm2bdw w <*> pr2bdw p
pr2bdw (Match x y p) = (\x y p->"(match "++x++" "++y++" "++p++")")
                        <$> tm2bdw x <*> tm2bdw y <*> pr2bdw p
pr2bdw (Plus p q) = (\p q->"(plus "++p++" "++q++")") <$> pr2bdw p <*> pr2bdw q
pr2bdw (Par p q) = (\p q->"(par "++p++" "++q++")") <$> pr2bdw p <*> pr2bdw q
pr2bdw (Nu b) = do (w,p) <- unbind b
                   (\w p->"(nu "++w++"\\"++p++")") <$> nm2bdw w <*> pr2bdw p

form2bdw :: Fresh m => Form -> m String
form2bdw FF = return "ff"
form2bdw TT = return "tt"
form2bdw (Conj []) = form2bdw TT
form2bdw (Conj [f]) = form2bdw f
form2bdw (Conj fs) = foldr1 (\x y -> "(conj "++x++" "++y++")") <$> mapM form2bdw fs
form2bdw (Disj []) = form2bdw FF
form2bdw (Disj [f]) = form2bdw f
form2bdw (Disj fs) = foldr1 (\x y -> "(disj "++x++" "++y++")") <$> mapM form2bdw fs
form2bdw (Dia a f) = (\a f -> "(diaAct "++a++" "++f++")") <$> act2bdw a <*> form2bdw f
form2bdw (Box a f) = (\a f -> "(boxAct "++a++" "++f++")") <$> act2bdw a <*> form2bdw f
form2bdw (DiaB (UpB x) b) = do (w,f) <- unbind b
                               (\x w f->"(diaOut "++x++" "++w++"\\"++f++")")
                                 <$> tm2bdw x <*> nm2bdw w <*> form2bdw f
form2bdw (DiaB (DnB x) b) = do (w,f) <- unbind b
                               (\x w f->"(diaInL "++x++" "++w++"\\"++f++")")
                                 <$> tm2bdw x <*> nm2bdw w <*> form2bdw f
form2bdw (BoxB (UpB x) b) = do (w,f) <- unbind b
                               (\x w f->"(boxOut "++x++" "++w++"\\"++f++")")
                                 <$> tm2bdw x <*> nm2bdw w <*> form2bdw f
form2bdw (BoxB (DnB x) b) = do (w,f) <- unbind b
                               (\x w f->"(boxIn "++x++" "++w++"\\"++f++")")
                                 <$> tm2bdw x <*> nm2bdw w <*> form2bdw f
form2bdw f@(DiaMatch [] _) = error (show f)
form2bdw (DiaMatch cs f) =
  foldr (\(x,y) f -> "(diaMatch "++x++" "++y++" "++f++")")
    <$> form2bdw f <*> sequence [(,)<$>tm2bdw x<*>tm2bdw y|(x,y)<-cs]
form2bdw f@(BoxMatch [] _) = error (show f)
form2bdw (BoxMatch cs f) =
  foldr (\(x,y) f -> "(boxMatch "++x++" "++y++" "++f++")")
    <$> form2bdw f <*> sequence [(,)<$>tm2bdw x<*>tm2bdw y|(x,y)<-cs]

In [8]:
x, y, z :: Nm
w = s2n "w"
x = s2n "x"
y = s2n "y"
z = s2n "z"

q1 = taup q2
q2 = (x.= y)(taup o)

p1 = tau .+ taup tau
p2 = inp x(z.\out x z o) .| out x y o
p3 = inp x(z.\out x z o) .| out y x o

axay = reverse [All x, All y]
axny = reverse [All x, Nab y]
nxay = reverse [Nab x, All y]
nxny = reverse [Nab x, Nab y]
axayazaw = reverse $ map All [x,y,z,w]

printLateOneFrom p = do
  putStrLn $ "one step from: " ++ render1line (pPrint p)
  mapM_ pp (runFreshMT (IdS.one p) :: [(Act,Pr)])


printOpenOneFrom nctx p = do
  putStrLn $ "one step from: " ++ show (reverse nctx) ++ " "++ render1line (pPrint p)
  mapM_ pp (runFreshMT (OpS.one nctx p) :: [([(Nm,Nm)],(Act,Pr))])

showForest = drawForest . map toTreeString
showTree = drawTree . toTreeString

render1line = renderStyle style{mode=OneLineMode}

toTreeString = foldTree (\log ts -> Node (render1line . pPrint $ log) ts)

axayaaab = map All [x,y,a,b]
a = s2n "a"
b = s2n "b"

ppp1 = taup(out a a o .+out b b o) .+ (x.= y) (taup $ out a a o)
qqq1 = taup(out a a o .+out b b o) .+ taup (out a a o)

ppp2 = nu$x.\out a x (inp a $y.\tau)
qqq2 = nu$x.\out a x (inp a $y.\(x.= y) tau)

t_tt = tau .+ taup tau

dosomething = do
  (s,(l,p')) <- OpS.one nctx p
  return $ subs nctx s p'
  where
  nctx = axayazaw
  p =
      (x.= w) . (z.= x) $
      -- (x.= y) . (w.= z) $
      taup (out x w o) .+ taup (out y z o)

dosomething2 = do
  (s,(l,bp')) <- OpS.oneb nctx p
  return $ subs nctx s bp'
  where
    nctx = []
    p = Nu$x.\(inp x$y.\taup o)

dosomething3 = do
  (s,(l,bp')) <- OpS.oneb nctx p
  return $ subs nctx s bp'
  where
    nctx = [All a]
    p = Nu$x.\ out a x o

dosomething4 = do
  (s,(l,p')) <- OpS.one nctx p
  return $ subs nctx s p'
  where
    nctx = [All a]
    p = Nu$x.\ out a x o

dosomething5 = do
  (s,(l,bp')) <- OpS.oneb nctx p
  return $ subs nctx s bp'
  where
    nctx = [All a]
    p = Nu$x.\ out x a o

### Late LTS

In [9]:
printLateOneFrom p1

one step from: (TauP Null) `Plus` (TauP (TauP Null))
(Tau, Null)
(Tau, TauP Null)

In [10]:
printLateOneFrom q1

one step from: TauP (Match (Var x) (Var y) (TauP Null))
(Tau, Match (Var x) (Var y) (TauP Null))

In [11]:
printLateOneFrom q2

one step from: Match (Var x) (Var y) (TauP Null)

In [12]:
printLateOneFrom p2

one step from: (In (Var x) (z.\(Out (Var x) (Var z) Null))) `Par` (Out (Var x) (Var y) Null)
(Up (Var x) (Var y),
 (In (Var x) (z.\(Out (Var x) (Var z) Null))) `Par` Null)
(Tau, (Out (Var x) (Var y) Null) `Par` Null)

In [13]:
printLateOneFrom p3

one step from: (In (Var x) (z.\(Out (Var x) (Var z) Null))) `Par` (Out (Var y) (Var x) Null)
(Up (Var y) (Var x),
 (In (Var x) (z.\(Out (Var x) (Var z) Null))) `Par` Null)

### Open LTS

In [14]:
printOpenOneFrom axay p1

one step from: [All x,All y] (TauP Null) `Plus` (TauP (TauP Null))
([], (Tau, Null))
([], (Tau, TauP Null))

In [15]:
printOpenOneFrom axay q1

one step from: [All x,All y] TauP (Match (Var x) (Var y) (TauP Null))
([], (Tau, Match (Var x) (Var y) (TauP Null)))

In [16]:
printOpenOneFrom axay q2

one step from: [All x,All y] Match (Var x) (Var y) (TauP Null)
([(y, x)], (Tau, Null))

In [17]:
printOpenOneFrom axny q2

one step from: [All x,Nab y] Match (Var x) (Var y) (TauP Null)

In [18]:
printOpenOneFrom nxay q2

one step from: [Nab x,All y] Match (Var x) (Var y) (TauP Null)
([(y, x)], (Tau, Null))

In [19]:
printOpenOneFrom nxny q2

one step from: [Nab x,Nab y] Match (Var x) (Var y) (TauP Null)

In [20]:
printOpenOneFrom axay p2

one step from: [All x,All y] (In (Var x) (z.\(Out (Var x) (Var z) Null))) `Par` (Out (Var x) (Var y) Null)
([],
 (Up (Var x) (Var y),
  (In (Var x) (z.\(Out (Var x) (Var z) Null))) `Par` Null))
([], (Tau, (Out (Var x) (Var y) Null) `Par` Null))

In [21]:
printOpenOneFrom axay p3

one step from: [All x,All y] (In (Var x) (z.\(Out (Var x) (Var z) Null))) `Par` (Out (Var y) (Var x) Null)
([],
 (Up (Var y) (Var x),
  (In (Var x) (z.\(Out (Var x) (Var z) Null))) `Par` Null))
([(y, x)], (Tau, (Out (Var x) (Var x) Null) `Par` Null))

In [22]:
runFreshMT dosomething :: [Pr]

[Out (Var x) (Var x) Null,Out (Var y) (Var x) Null]

In [23]:
bisim axay (tau .+ taup tau) (TauP $ (x.= y)tau)

putStrLn . showForest $ bisim' axay (tau .+ taup tau) (TauP $ (x.= y) tau)

mapM_ print . forest2df $ bisim' axay (tau .+ taup tau) (taup $ (x.= y) tau)

False

Left (One [All y, All x] [] Tau Null)
|
`- Right (One [All y, All x] [] Tau (Match (Var x) (Var y) (TauP Null)))
   |
   `- Right (One [All y, All x] [(y, x)] Tau Null)

Left (One [All y, All x] [] Tau (TauP Null))
|
`- Right (One [All y, All x] [] Tau (Match (Var x) (Var y) (TauP Null)))
   |
   +- Left (One [All y, All x] [] Tau Null)
   |
   `- Right (One [All y, All x] [(y, x)] Tau Null)
      |
      `- Left (One [All y, All x] [(y, x)] Tau Null)

Right (One [All y, All x] [] Tau (Match (Var x) (Var y) (TauP Null)))
|
+- Left (One [All y, All x] [] Tau Null)
|  |
|  `- Right (One [All y, All x] [(y, x)] Tau Null)
|
`- Left (One [All y, All x] [] Tau (TauP Null))
   |
   +- Left (One [All y, All x] [] Tau Null)
   |
   `- Right (One [All y, All x] [(y, x)] Tau Null)
      |
      `- Left (One [All y, All x] [(y, x)] Tau Null)

(Dia Tau (Box Tau FF),Box Tau (BoxMatch [(Var y,Var x)] (Dia Tau TT)))
(Dia Tau (Dia Tau TT),Box Tau (Box Tau (DiaMatch [(Var y,Var x)] TT)))
(Box Tau (Disj [Box Tau FF,Dia Tau TT]),Dia Tau (Conj [BoxMatch [(Var y,Var x)] (Dia Tau TT),Box Tau (DiaMatch [(Var y,Var x)] TT)]))

In [24]:
bisim axay (taup $ (x.= y) tau) (tau .+ taup tau)
putStrLn . showForest $ bisim' axay (taup $ (x.= y) tau) (tau .+ taup tau)
mapM_ pp . forest2df $ bisim' axay (taup $ (x.= y)tau) (tau .+ taup tau)

False

Left (One [All y, All x] [] Tau (Match (Var x) (Var y) (TauP Null)))
|
+- Right (One [All y, All x] [] Tau Null)
|  |
|  `- Left (One [All y, All x] [(y, x)] Tau Null)
|
`- Right (One [All y, All x] [] Tau (TauP Null))
   |
   +- Left (One [All y, All x] [(y, x)] Tau Null)
   |  |
   |  `- Right (One [All y, All x] [(y, x)] Tau Null)
   |
   `- Right (One [All y, All x] [] Tau Null)

Right (One [All y, All x] [] Tau Null)
|
`- Left (One [All y, All x] [] Tau (Match (Var x) (Var y) (TauP Null)))
   |
   `- Left (One [All y, All x] [(y, x)] Tau Null)

Right (One [All y, All x] [] Tau (TauP Null))
|
`- Left (One [All y, All x] [] Tau (Match (Var x) (Var y) (TauP Null)))
   |
   +- Left (One [All y, All x] [(y, x)] Tau Null)
   |  |
   |  `- Right (One [All y, All x] [(y, x)] Tau Null)
   |
   `- Right (One [All y, All x] [] Tau Null)

(Dia Tau (Conj[BoxMatch [(Var y, Var x)] (Dia Tau TT),
               Box Tau (DiaMatch [(Var y, Var x)] TT)]),
 Box Tau (Disj[Box Tau FF, Dia Tau TT]))
(Box Tau (BoxMatch [(Var y, Var x)] (Dia Tau TT)),
 Dia Tau (Box Tau FF))
(Box Tau (Box Tau (DiaMatch [(Var y, Var x)] TT)),
 Dia Tau (Dia Tau TT))

In [25]:
print $ bisim axay (taup ((x.= y) tau) .+ t_tt) t_tt
putStrLn . showForest $ bisim' axay (taup ((x.= y) tau) .+ t_tt) t_tt
mapM_ pp . forest2df $ bisim' axay (taup ((x.= y) tau) .+ t_tt) t_tt

False

Left (One [All y, All x] [] Tau (Match (Var x) (Var y) (TauP Null)))
|
+- Right (One [All y, All x] [] Tau Null)
|  |
|  `- Left (One [All y, All x] [(y, x)] Tau Null)
|
`- Right (One [All y, All x] [] Tau (TauP Null))
   |
   +- Left (One [All y, All x] [(y, x)] Tau Null)
   |  |
   |  `- Right (One [All y, All x] [(y, x)] Tau Null)
   |
   `- Right (One [All y, All x] [] Tau Null)

Left (One [All y, All x] [] Tau Null)
|
+- Right (One [All y, All x] [] Tau Null)
|
`- Right (One [All y, All x] [] Tau (TauP Null))
   |
   `- Right (One [All y, All x] [] Tau Null)

Left (One [All y, All x] [] Tau (TauP Null))
|
+- Right (One [All y, All x] [] Tau Null)
|  |
|  `- Left (One [All y, All x] [] Tau Null)
|
`- Right (One [All y, All x] [] Tau (TauP Null))
   |
   +- Left (One [All y, All x] [] Tau Null)
   |  |
   |  `- Right (One [All y, All x] [] Tau Null)
   |
   `- Right (One [All y, All x] [] Tau Null)
      |
      `- Left (One [All y, All x] [] Tau Null)

Right (One [All y, All x] [] 

(Dia Tau (Conj[BoxMatch [(Var y, Var x)] (Dia Tau TT),
               Box Tau (DiaMatch [(Var y, Var x)] TT)]),
 Box Tau (Disj[Box Tau FF, Dia Tau TT]))

In [26]:
  putStrLn . showForest $ bisim' axay (tau .+ taup tau) (taup $ (x.= y) tau)
  mapM_ pp . forest2df $ bisim' axay (tau .+ taup tau) (taup $ (x.= y) tau)

Left (One [All y, All x] [] Tau Null)
|
`- Right (One [All y, All x] [] Tau (Match (Var x) (Var y) (TauP Null)))
   |
   `- Right (One [All y, All x] [(y, x)] Tau Null)

Left (One [All y, All x] [] Tau (TauP Null))
|
`- Right (One [All y, All x] [] Tau (Match (Var x) (Var y) (TauP Null)))
   |
   +- Left (One [All y, All x] [] Tau Null)
   |
   `- Right (One [All y, All x] [(y, x)] Tau Null)
      |
      `- Left (One [All y, All x] [(y, x)] Tau Null)

Right (One [All y, All x] [] Tau (Match (Var x) (Var y) (TauP Null)))
|
+- Left (One [All y, All x] [] Tau Null)
|  |
|  `- Right (One [All y, All x] [(y, x)] Tau Null)
|
`- Left (One [All y, All x] [] Tau (TauP Null))
   |
   +- Left (One [All y, All x] [] Tau Null)
   |
   `- Right (One [All y, All x] [(y, x)] Tau Null)
      |
      `- Left (One [All y, All x] [(y, x)] Tau Null)

(Dia Tau (Box Tau FF),
 Box Tau (BoxMatch [(Var y, Var x)] (Dia Tau TT)))
(Dia Tau (Dia Tau TT),
 Box Tau (Box Tau (DiaMatch [(Var y, Var x)] TT)))
(Box Tau (Disj[Box Tau FF, Dia Tau TT]),
 Dia Tau (Conj[BoxMatch [(Var y, Var x)] (Dia Tau TT),
               Box Tau (DiaMatch [(Var y, Var x)] TT)]))

In [27]:
  putStrLn . showForest $ bisim' [All x] (inp x$z.\tau .+ tau) (inp x$z.\tau .+ out z x o)
  mapM_ pp . forest2df $ bisim' [All x] (inp x$z.\tau .+ tau) (inp x$z.\tau .+ out z x o)

Left (OneB [All x] [] (DnB (Var x)) (z.\((TauP Null) `Plus` (TauP Null))))
|
`- Right (OneB [All x] [] (DnB (Var x)) (z.\((TauP Null) `Plus` (Out (Var z) (Var x) Null))))
   |
   +- Left (One [All z1, All x] [] Tau Null)
   |  |
   |  `- Right (One [All z1, All x] [] Tau Null)
   |
   +- Left (One [All z1, All x] [] Tau Null)
   |  |
   |  `- Right (One [All z1, All x] [] Tau Null)
   |
   +- Right (One [All z1, All x] [] Tau Null)
   |  |
   |  +- Left (One [All z1, All x] [] Tau Null)
   |  |
   |  `- Left (One [All z1, All x] [] Tau Null)
   |
   `- Right (One [All z1, All x] [] (Up (Var z1) (Var x)) Null)

Right (OneB [All x] [] (DnB (Var x)) (z.\((TauP Null) `Plus` (Out (Var z) (Var x) Null))))
|
`- Left (OneB [All x] [] (DnB (Var x)) (z.\((TauP Null) `Plus` (TauP Null))))
   |
   +- Left (One [All z1, All x] [] Tau Null)
   |  |
   |  `- Right (One [All z1, All x] [] Tau Null)
   |
   +- Left (One [All z1, All x] [] Tau Null)
   |  |
   |  `- Right (One [All z1, All x] [] Tau Nul

(DiaB (DnB (Var x)) (z.\(Box (Up (Var z) (Var x)) FF)),
 BoxB (DnB (Var x)) (z.\(Dia (Up (Var z) (Var x)) TT)))
(BoxB (DnB (Var x)) (z.\(Box (Up (Var z) (Var x)) FF)),
 DiaB (DnB (Var x)) (z.\(Dia (Up (Var z) (Var x)) TT)))

In [28]:
putStrLn . showForest $ bisim' axayazaw ((z.= w) tau) ((x.= y) tau)
mapM_ pp . forest2df $ bisim' axayazaw ((z.= w) tau) ((x.= y) tau)

Left (One [All w, All z, All y, All x] [(w, z)] Tau Null)

Right (One [All w, All z, All y, All x] [(y, x)] Tau Null)

(BoxMatch [(Var w, Var z)] (Dia Tau TT),
 Box Tau (DiaMatch [(Var y, Var x)] TT))
(Box Tau (DiaMatch [(Var w, Var z)] TT),
 BoxMatch [(Var y, Var x)] (Dia Tau TT))

In [29]:
bisim axayaaab ppp1 qqq1
putStrLn . showForest $ bisim' axayaaab ppp1 qqq1
mapM_ pp . forest2df $ bisim' axayaaab ppp1 qqq1

False

Left (One [All x, All y, All a, All b] [] Tau ((Out (Var a) (Var a) Null) `Plus` (Out (Var b) (Var b) Null)))
|
+- Right (One [All x, All y, All a, All b] [] Tau ((Out (Var a) (Var a) Null) `Plus` (Out (Var b) (Var b) Null)))
|  |
|  +- Left (One [All x, All y, All a, All b] [] (Up (Var a) (Var a)) Null)
|  |  |
|  |  `- Right (One [All x, All y, All a, All b] [] (Up (Var a) (Var a)) Null)
|  |
|  +- Left (One [All x, All y, All a, All b] [] (Up (Var b) (Var b)) Null)
|  |  |
|  |  `- Right (One [All x, All y, All a, All b] [] (Up (Var b) (Var b)) Null)
|  |
|  +- Right (One [All x, All y, All a, All b] [] (Up (Var a) (Var a)) Null)
|  |  |
|  |  `- Left (One [All x, All y, All a, All b] [] (Up (Var a) (Var a)) Null)
|  |
|  `- Right (One [All x, All y, All a, All b] [] (Up (Var b) (Var b)) Null)
|     |
|     `- Left (One [All x, All y, All a, All b] [] (Up (Var b) (Var b)) Null)
|
`- Right (One [All x, All y, All a, All b] [] Tau (Out (Var a) (Var a) Null))
   |
   +- Left (One [All 

(Box Tau (Disj[DiaMatch [(Var x, Var y)] TT,
               Dia (Up (Var b) (Var b)) TT]),
 Dia Tau (Box (Up (Var b) (Var b)) FF))

In [30]:
bisim [All a] ppp2 qqq2
putStrLn . showForest $ bisim' [All a] ppp2 qqq2
mapM_ pp . forest2df $ bisim' [All a] ppp2 qqq2

False

Left (OneB [All a] [] (UpB (Var a)) (x.\(In (Var a) (y.\(TauP Null)))))
|
`- Right (OneB [All a] [] (UpB (Var a)) (x.\(In (Var a) (y.\(Match (Var x) (Var y) (TauP Null))))))
   |
   +- Left (OneB [Nab x1, All a] [] (DnB (Var a)) (y.\(TauP Null)))
   |  |
   |  `- Right (OneB [Nab x1, All a] [] (DnB (Var a)) (y.\(Match (Var x1) (Var y) (TauP Null))))
   |     |
   |     +- Left (One [All y2, Nab x1, All a] [] Tau Null)
   |     |
   |     `- Right (One [All y2, Nab x1, All a] [(x1, a)] Tau Null)
   |        |
   |        `- Left (One [All y2, Nab x1, All a] [(x1, a)] Tau Null)
   |
   `- Right (OneB [Nab x1, All a] [] (DnB (Var a)) (y.\(Match (Var x1) (Var y) (TauP Null))))
      |
      `- Left (OneB [Nab x1, All a] [] (DnB (Var a)) (y.\(TauP Null)))
         |
         +- Left (One [All y2, Nab x1, All a] [] Tau Null)
         |
         `- Right (One [All y2, Nab x1, All a] [(x1, a)] Tau Null)
            |
            `- Left (One [All y2, Nab x1, All a] [(x1, a)] Tau Null)

Right (

(DiaB (UpB (Var a)) (x.\(DiaB (DnB (Var a)) (y.\(Dia Tau TT)))),
 BoxB (UpB (Var a)) (x.\(BoxB (DnB (Var a)) (y.\(Box Tau (DiaMatch [(Var x,
                                                                     Var a)] TT))))))
(DiaB (UpB (Var a)) (x.\(BoxB (DnB (Var a)) (y.\(Dia Tau TT)))),
 BoxB (UpB (Var a)) (x.\(DiaB (DnB (Var a)) (y.\(Box Tau (DiaMatch [(Var x,
                                                                     Var a)] TT))))))
(BoxB (UpB (Var a)) (x.\(DiaB (DnB (Var a)) (y.\(Dia Tau TT)))),
 DiaB (UpB (Var a)) (x.\(BoxB (DnB (Var a)) (y.\(Box Tau (DiaMatch [(Var x,
                                                                     Var a)] TT))))))
(BoxB (UpB (Var a)) (x.\(BoxB (DnB (Var a)) (y.\(Dia Tau TT)))),
 DiaB (UpB (Var a)) (x.\(DiaB (DnB (Var a)) (y.\(Box Tau (DiaMatch [(Var x,
                                                                     Var a)] TT))))))

In [31]:
pp (runFreshMT dosomething2 :: [PrB])
pp (runFreshMT dosomething3 :: [PrB])
pp (runFreshMT dosomething4 :: [Pr])
pp (runFreshMT dosomething5 :: [PrB])

[]

[x.\Null]

[]

[]

In [32]:
bisim axay ((x.= y) (taup tau) .+ tau) (taup tau .+ tau)
putStrLn . showForest $ bisim' axay ((x.= y) (taup tau) .+ tau) (taup tau .+ tau)
mapM_ pp . forest2df $ bisim' axay ((x.= y) (taup tau) .+ tau) (taup tau .+ tau)

False

Left (One [All y, All x] [(y, x)] Tau (TauP Null))
|
+- Right (One [All y, All x] [(y, x)] Tau (TauP Null))
|  |
|  +- Left (One [All y, All x] [] Tau Null)
|  |  |
|  |  `- Right (One [All y, All x] [] Tau Null)
|  |
|  `- Right (One [All y, All x] [] Tau Null)
|     |
|     `- Left (One [All y, All x] [] Tau Null)
|
`- Right (One [All y, All x] [(y, x)] Tau Null)
   |
   `- Left (One [All y, All x] [] Tau Null)

Left (One [All y, All x] [] Tau Null)
|
+- Right (One [All y, All x] [] Tau (TauP Null))
|  |
|  `- Right (One [All y, All x] [] Tau Null)
|
`- Right (One [All y, All x] [] Tau Null)

Right (One [All y, All x] [] Tau (TauP Null))
|
`- Left (One [All y, All x] [] Tau Null)
   |
   `- Right (One [All y, All x] [] Tau Null)

Right (One [All y, All x] [] Tau Null)
|
`- Left (One [All y, All x] [] Tau Null)

(Box Tau (Disj[DiaMatch [(Var y, Var x)] TT, Box Tau FF]),
 Dia Tau (Dia Tau TT))

In [33]:
bisim axay (taup ((x.= y) tau) .+ t_tt) t_tt
putStrLn . showForest $ bisim' axay (taup ((x.= y) tau) .+ t_tt) t_tt
mapM_ pp . forest2df $ bisim' axay (taup ((x.= y) tau) .+ t_tt) t_tt

False

Left (One [All y, All x] [] Tau (Match (Var x) (Var y) (TauP Null)))
|
+- Right (One [All y, All x] [] Tau Null)
|  |
|  `- Left (One [All y, All x] [(y, x)] Tau Null)
|
`- Right (One [All y, All x] [] Tau (TauP Null))
   |
   +- Left (One [All y, All x] [(y, x)] Tau Null)
   |  |
   |  `- Right (One [All y, All x] [(y, x)] Tau Null)
   |
   `- Right (One [All y, All x] [] Tau Null)

Left (One [All y, All x] [] Tau Null)
|
+- Right (One [All y, All x] [] Tau Null)
|
`- Right (One [All y, All x] [] Tau (TauP Null))
   |
   `- Right (One [All y, All x] [] Tau Null)

Left (One [All y, All x] [] Tau (TauP Null))
|
+- Right (One [All y, All x] [] Tau Null)
|  |
|  `- Left (One [All y, All x] [] Tau Null)
|
`- Right (One [All y, All x] [] Tau (TauP Null))
   |
   +- Left (One [All y, All x] [] Tau Null)
   |  |
   |  `- Right (One [All y, All x] [] Tau Null)
   |
   `- Right (One [All y, All x] [] Tau Null)
      |
      `- Left (One [All y, All x] [] Tau Null)

Right (One [All y, All x] [] 

(Dia Tau (Conj[BoxMatch [(Var y, Var x)] (Dia Tau TT),
               Box Tau (DiaMatch [(Var y, Var x)] TT)]),
 Box Tau (Disj[Box Tau FF, Dia Tau TT]))

In [34]:
 putStrLn . showForest $ bisim' axay (tau .+ taup tau) (taup $ (x.= y) tau)
 mapM_ pp . forest2df $ bisim' axay (tau .+ taup tau) (taup $ (x.= y) tau)

Left (One [All y, All x] [] Tau Null)
|
`- Right (One [All y, All x] [] Tau (Match (Var x) (Var y) (TauP Null)))
   |
   `- Right (One [All y, All x] [(y, x)] Tau Null)

Left (One [All y, All x] [] Tau (TauP Null))
|
`- Right (One [All y, All x] [] Tau (Match (Var x) (Var y) (TauP Null)))
   |
   +- Left (One [All y, All x] [] Tau Null)
   |
   `- Right (One [All y, All x] [(y, x)] Tau Null)
      |
      `- Left (One [All y, All x] [(y, x)] Tau Null)

Right (One [All y, All x] [] Tau (Match (Var x) (Var y) (TauP Null)))
|
+- Left (One [All y, All x] [] Tau Null)
|  |
|  `- Right (One [All y, All x] [(y, x)] Tau Null)
|
`- Left (One [All y, All x] [] Tau (TauP Null))
   |
   +- Left (One [All y, All x] [] Tau Null)
   |
   `- Right (One [All y, All x] [(y, x)] Tau Null)
      |
      `- Left (One [All y, All x] [(y, x)] Tau Null)

(Dia Tau (Box Tau FF),
 Box Tau (BoxMatch [(Var y, Var x)] (Dia Tau TT)))
(Dia Tau (Dia Tau TT),
 Box Tau (Box Tau (DiaMatch [(Var y, Var x)] TT)))
(Box Tau (Disj[Box Tau FF, Dia Tau TT]),
 Dia Tau (Conj[BoxMatch [(Var y, Var x)] (Dia Tau TT),
               Box Tau (DiaMatch [(Var y, Var x)] TT)]))

In [35]:
putStrLn . showForest $ bisim' [All x] (inp x$z.\tau .+ tau) (inp x$z.\tau .+ out z x o)
mapM_ pp . forest2df $ bisim' [All x] (inp x$z.\tau .+ tau) (inp x$z.\tau .+ out z x o)

Left (OneB [All x] [] (DnB (Var x)) (z.\((TauP Null) `Plus` (TauP Null))))
|
`- Right (OneB [All x] [] (DnB (Var x)) (z.\((TauP Null) `Plus` (Out (Var z) (Var x) Null))))
   |
   +- Left (One [All z1, All x] [] Tau Null)
   |  |
   |  `- Right (One [All z1, All x] [] Tau Null)
   |
   +- Left (One [All z1, All x] [] Tau Null)
   |  |
   |  `- Right (One [All z1, All x] [] Tau Null)
   |
   +- Right (One [All z1, All x] [] Tau Null)
   |  |
   |  +- Left (One [All z1, All x] [] Tau Null)
   |  |
   |  `- Left (One [All z1, All x] [] Tau Null)
   |
   `- Right (One [All z1, All x] [] (Up (Var z1) (Var x)) Null)

Right (OneB [All x] [] (DnB (Var x)) (z.\((TauP Null) `Plus` (Out (Var z) (Var x) Null))))
|
`- Left (OneB [All x] [] (DnB (Var x)) (z.\((TauP Null) `Plus` (TauP Null))))
   |
   +- Left (One [All z1, All x] [] Tau Null)
   |  |
   |  `- Right (One [All z1, All x] [] Tau Null)
   |
   +- Left (One [All z1, All x] [] Tau Null)
   |  |
   |  `- Right (One [All z1, All x] [] Tau Nul

(DiaB (DnB (Var x)) (z.\(Box (Up (Var z) (Var x)) FF)),
 BoxB (DnB (Var x)) (z.\(Dia (Up (Var z) (Var x)) TT)))
(BoxB (DnB (Var x)) (z.\(Box (Up (Var z) (Var x)) FF)),
 DiaB (DnB (Var x)) (z.\(Dia (Up (Var z) (Var x)) TT)))

In [36]:
putStrLn . showForest $ bisim' axayazaw ((z.= w) tau) ((x.= y) tau)
mapM_ pp . forest2df $ bisim' axayazaw ((z.= w) tau) ((x.= y) tau)

Left (One [All w, All z, All y, All x] [(w, z)] Tau Null)

Right (One [All w, All z, All y, All x] [(y, x)] Tau Null)

(BoxMatch [(Var w, Var z)] (Dia Tau TT),
 Box Tau (DiaMatch [(Var y, Var x)] TT))
(Box Tau (DiaMatch [(Var w, Var z)] TT),
 BoxMatch [(Var y, Var x)] (Dia Tau TT))

In [37]:
bisim axayaaab ppp1 qqq1
putStrLn . showForest $ bisim' axayaaab ppp1 qqq1
mapM_ pp . forest2df $ bisim' axayaaab ppp1 qqq1

False

Left (One [All x, All y, All a, All b] [] Tau ((Out (Var a) (Var a) Null) `Plus` (Out (Var b) (Var b) Null)))
|
+- Right (One [All x, All y, All a, All b] [] Tau ((Out (Var a) (Var a) Null) `Plus` (Out (Var b) (Var b) Null)))
|  |
|  +- Left (One [All x, All y, All a, All b] [] (Up (Var a) (Var a)) Null)
|  |  |
|  |  `- Right (One [All x, All y, All a, All b] [] (Up (Var a) (Var a)) Null)
|  |
|  +- Left (One [All x, All y, All a, All b] [] (Up (Var b) (Var b)) Null)
|  |  |
|  |  `- Right (One [All x, All y, All a, All b] [] (Up (Var b) (Var b)) Null)
|  |
|  +- Right (One [All x, All y, All a, All b] [] (Up (Var a) (Var a)) Null)
|  |  |
|  |  `- Left (One [All x, All y, All a, All b] [] (Up (Var a) (Var a)) Null)
|  |
|  `- Right (One [All x, All y, All a, All b] [] (Up (Var b) (Var b)) Null)
|     |
|     `- Left (One [All x, All y, All a, All b] [] (Up (Var b) (Var b)) Null)
|
`- Right (One [All x, All y, All a, All b] [] Tau (Out (Var a) (Var a) Null))
   |
   +- Left (One [All 

(Box Tau (Disj[DiaMatch [(Var x, Var y)] TT,
               Dia (Up (Var b) (Var b)) TT]),
 Dia Tau (Box (Up (Var b) (Var b)) FF))

In [38]:
bisim [All a] ppp2 qqq2
putStrLn . showForest $ bisim' [All a] ppp2 qqq2
mapM_ pp . forest2df $ bisim' [All a] ppp2 qqq2

False

Left (OneB [All a] [] (UpB (Var a)) (x.\(In (Var a) (y.\(TauP Null)))))
|
`- Right (OneB [All a] [] (UpB (Var a)) (x.\(In (Var a) (y.\(Match (Var x) (Var y) (TauP Null))))))
   |
   +- Left (OneB [Nab x1, All a] [] (DnB (Var a)) (y.\(TauP Null)))
   |  |
   |  `- Right (OneB [Nab x1, All a] [] (DnB (Var a)) (y.\(Match (Var x1) (Var y) (TauP Null))))
   |     |
   |     +- Left (One [All y2, Nab x1, All a] [] Tau Null)
   |     |
   |     `- Right (One [All y2, Nab x1, All a] [(x1, a)] Tau Null)
   |        |
   |        `- Left (One [All y2, Nab x1, All a] [(x1, a)] Tau Null)
   |
   `- Right (OneB [Nab x1, All a] [] (DnB (Var a)) (y.\(Match (Var x1) (Var y) (TauP Null))))
      |
      `- Left (OneB [Nab x1, All a] [] (DnB (Var a)) (y.\(TauP Null)))
         |
         +- Left (One [All y2, Nab x1, All a] [] Tau Null)
         |
         `- Right (One [All y2, Nab x1, All a] [(x1, a)] Tau Null)
            |
            `- Left (One [All y2, Nab x1, All a] [(x1, a)] Tau Null)

Right (

(DiaB (UpB (Var a)) (x.\(DiaB (DnB (Var a)) (y.\(Dia Tau TT)))),
 BoxB (UpB (Var a)) (x.\(BoxB (DnB (Var a)) (y.\(Box Tau (DiaMatch [(Var x,
                                                                     Var a)] TT))))))
(DiaB (UpB (Var a)) (x.\(BoxB (DnB (Var a)) (y.\(Dia Tau TT)))),
 BoxB (UpB (Var a)) (x.\(DiaB (DnB (Var a)) (y.\(Box Tau (DiaMatch [(Var x,
                                                                     Var a)] TT))))))
(BoxB (UpB (Var a)) (x.\(DiaB (DnB (Var a)) (y.\(Dia Tau TT)))),
 DiaB (UpB (Var a)) (x.\(BoxB (DnB (Var a)) (y.\(Box Tau (DiaMatch [(Var x,
                                                                     Var a)] TT))))))
(BoxB (UpB (Var a)) (x.\(BoxB (DnB (Var a)) (y.\(Dia Tau TT)))),
 DiaB (UpB (Var a)) (x.\(DiaB (DnB (Var a)) (y.\(Box Tau (DiaMatch [(Var x,
                                                                     Var a)] TT))))))

In [39]:
pp (runFreshMT dosomething2 :: [PrB])
pp (runFreshMT dosomething3 :: [PrB])
pp (runFreshMT dosomething4 :: [Pr])
pp (runFreshMT dosomething5 :: [PrB])

[]

[x.\Null]

[]

[]

In [40]:
bisim axay ((x.= y) (taup tau) .+ tau) (taup tau .+ tau)
putStrLn . showForest $ bisim' axay ((x.= y) (taup tau) .+ tau) (taup tau .+ tau)
mapM_ pp . forest2df $ bisim' axay ((x.= y) (taup tau) .+ tau) (taup tau .+ tau)

False

Left (One [All y, All x] [(y, x)] Tau (TauP Null))
|
+- Right (One [All y, All x] [(y, x)] Tau (TauP Null))
|  |
|  +- Left (One [All y, All x] [] Tau Null)
|  |  |
|  |  `- Right (One [All y, All x] [] Tau Null)
|  |
|  `- Right (One [All y, All x] [] Tau Null)
|     |
|     `- Left (One [All y, All x] [] Tau Null)
|
`- Right (One [All y, All x] [(y, x)] Tau Null)
   |
   `- Left (One [All y, All x] [] Tau Null)

Left (One [All y, All x] [] Tau Null)
|
+- Right (One [All y, All x] [] Tau (TauP Null))
|  |
|  `- Right (One [All y, All x] [] Tau Null)
|
`- Right (One [All y, All x] [] Tau Null)

Right (One [All y, All x] [] Tau (TauP Null))
|
`- Left (One [All y, All x] [] Tau Null)
   |
   `- Right (One [All y, All x] [] Tau Null)

Right (One [All y, All x] [] Tau Null)
|
`- Left (One [All y, All x] [] Tau Null)

(Box Tau (Disj[DiaMatch [(Var y, Var x)] TT, Box Tau FF]),
 Dia Tau (Dia Tau TT))

In [41]:
bisim axay (taup tau .+ tau) ((x.= y) (taup tau) .+ tau)
putStrLn . showForest $ bisim' axay (taup tau .+ tau) ((x.= y) (taup tau) .+ tau)
mapM_ pp . forest2df $ bisim' axay (taup tau .+ tau) ((x.= y) (taup tau) .+ tau)

False

Left (One [All y, All x] [] Tau (TauP Null))
|
`- Right (One [All y, All x] [] Tau Null)
   |
   `- Left (One [All y, All x] [] Tau Null)

Left (One [All y, All x] [] Tau Null)
|
`- Right (One [All y, All x] [] Tau Null)

Right (One [All y, All x] [(y, x)] Tau (TauP Null))
|
+- Left (One [All y, All x] [(y, x)] Tau (TauP Null))
|  |
|  +- Left (One [All y, All x] [] Tau Null)
|  |  |
|  |  `- Right (One [All y, All x] [] Tau Null)
|  |
|  `- Right (One [All y, All x] [] Tau Null)
|     |
|     `- Left (One [All y, All x] [] Tau Null)
|
`- Left (One [All y, All x] [(y, x)] Tau Null)
   |
   `- Right (One [All y, All x] [] Tau Null)

Right (One [All y, All x] [] Tau Null)
|
+- Left (One [All y, All x] [] Tau (TauP Null))
|  |
|  `- Left (One [All y, All x] [] Tau Null)
|
`- Left (One [All y, All x] [] Tau Null)

(Dia Tau (Dia Tau TT),
 Box Tau (Disj[DiaMatch [(Var y, Var x)] TT, Box Tau FF]))

In [42]:
bisim [All a] (Nu$b.\out a b (inp a $x.\(x.= b) (out x x o))) (Nu$b.\out a b (inp a $x.\out x x o))
putStrLn . showForest $ bisim' [All a] (Nu$b.\out a b (inp a $x.\(x.= b) (out x x o))) (Nu$b.\out a b (inp a $x.\out x x o))
mapM_ pp . forest2df $ bisim' [All a] (Nu$b.\out a b (inp a $x.\(x.= b) (out x x o))) (Nu$b.\out a b (inp a $x.\out x x o))

False

Left (OneB [All a] [] (UpB (Var a)) (b.\(In (Var a) (x.\(Match (Var x) (Var b) (Out (Var x) (Var x) Null))))))
|
`- Right (OneB [All a] [] (UpB (Var a)) (b.\(In (Var a) (x.\(Out (Var x) (Var x) Null)))))
   |
   +- Left (OneB [Nab b1, All a] [] (DnB (Var a)) (x.\(Match (Var x) (Var b1) (Out (Var x) (Var x) Null))))
   |  |
   |  `- Right (OneB [Nab b1, All a] [] (DnB (Var a)) (x.\(Out (Var x) (Var x) Null)))
   |     |
   |     +- Left (One [All x2, Nab b1, All a] [(b1, a)] (Up (Var x2) (Var x2)) Null)
   |     |  |
   |     |  `- Right (One [All x2, Nab b1, All a] [(b1, a)] (Up (Var x2) (Var x2)) Null)
   |     |
   |     `- Right (One [All x2, Nab b1, All a] [] (Up (Var x2) (Var x2)) Null)
   |
   `- Right (OneB [Nab b1, All a] [] (DnB (Var a)) (x.\(Out (Var x) (Var x) Null)))
      |
      `- Left (OneB [Nab b1, All a] [] (DnB (Var a)) (x.\(Match (Var x) (Var b1) (Out (Var x) (Var x) Null))))
         |
         +- Left (One [All x2, Nab b1, All a] [(b1, a)] (Up (Var x2) (Var x2)) N

(DiaB (UpB (Var a)) (b.\(DiaB (DnB (Var a)) (x.\(Box (Up (Var x) (Var x)) (DiaMatch [(Var b,
                                                                                      Var a)] TT))))),
 BoxB (UpB (Var a)) (b.\(BoxB (DnB (Var a)) (x.\(Dia (Up (Var x) (Var x)) TT)))))
(DiaB (UpB (Var a)) (b.\(BoxB (DnB (Var a)) (x.\(Box (Up (Var x) (Var x)) (DiaMatch [(Var b,
                                                                                      Var a)] TT))))),
 BoxB (UpB (Var a)) (b.\(DiaB (DnB (Var a)) (x.\(Dia (Up (Var x) (Var x)) TT)))))
(BoxB (UpB (Var a)) (b.\(DiaB (DnB (Var a)) (x.\(Box (Up (Var x) (Var x)) (DiaMatch [(Var b,
                                                                                      Var a)] TT))))),
 DiaB (UpB (Var a)) (b.\(BoxB (DnB (Var a)) (x.\(Dia (Up (Var x) (Var x)) TT)))))
(BoxB (UpB (Var a)) (b.\(BoxB (DnB (Var a)) (x.\(Box (Up (Var x) (Var x)) (DiaMatch [(Var b,
                                                                         

# some quick things TODO
exploit the HTML environment

#### Write pretty print to LaTeX format and let it be displayed using HTML + MathJAX.
 

In [43]:
p1  -- for example, instead of output below

Plus (TauP Null) (TauP (TauP Null))

In [44]:
-- generate something like this from p1, and also for processes 
import IHaskell.Display
html "$\\tau.0 + \\tau.\\tau.0$"

$\tau.0 + \tau.\tau.0$

#### ditto on Tree view
A there is a library for Data.Tree to html output

# some not so quick things TODO

 1. refactoring to reduce code ducplications
 1. manage inequality feature and other notions of bisimulation (early,quasiopen) seamlessy with a single codebase

these should be useful (almost necessary) thing to do if we want to work on an algorithm
that adapts from failure of open bisim to see whether quasi-open is acheivable from there.

And, there are also explorations on how to suppport additional features/variants of pi-calculus: applied-pi style terms, replication, etc